In [1]:
import librosa
import numpy as np
import pretty_midi
from moviepy.editor import *
import glob
import os

In [2]:
file_name = '../songs/guren.mid'

In [3]:
SRC_PATH = "../new_src"

In [4]:
os.path.join(SRC_PATH, "61.wav")

'../new_src\\61.wav'

In [5]:
os.path.join(SRC_PATH, "*")

'../new_src\\*'

In [6]:
glob.glob(SRC_PATH + "\\*")

['../new_src\\60.mp4']

In [7]:
os.path.basename(os.path.join(SRC_PATH, "61.wav"))

'61.wav'

In [8]:
midi_data = pretty_midi.PrettyMIDI(file_name)

In [9]:
notes = []

In [10]:
for ins in midi_data.instruments:
    notes += ins.notes

In [11]:
notes.sort(key=lambda x: x.start)

In [12]:
notes[:5]

[Note(start=2.655389, end=2.828041, pitch=61, velocity=93),
 Note(start=2.655389, end=3.321135, pitch=37, velocity=84),
 Note(start=2.655389, end=3.321135, pitch=49, velocity=99),
 Note(start=2.828041, end=2.983428, pitch=61, velocity=88),
 Note(start=2.993097, end=3.146412, pitch=64, velocity=89)]

In [13]:
def allNotes(notes):
    pitchs = {}
    for note in notes:
        pitchs[note.pitch] = True
    result = list(pitchs.keys())
    result.sort()
    return result

In [14]:
note = allNotes(notes)

In [15]:
note

[25,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 91,
 92]

In [16]:
def get_clip(note_number):
    path_clip = os.path.join(SRC_PATH, str(note_number) + ".mp4")
    clip = VideoFileClip(path_clip)
    return clip

In [20]:
def path_to_number(path, only_key = False):
    files = glob.glob(os.path.join(SRC_PATH, "*"))
    clips = {}
    for f in files:
        number = os.path.basename(f).split(".")[0]
        if only_key:
            clips[number] = f
        else:
            clips[number] = get_clip(number)
    return clips

In [21]:
clips = path_to_number(SRC_PATH, only_key=True)

In [22]:
clips

{'60': '../new_src\\60.mp4'}

In [23]:
noteClip = list(clips.keys())

In [24]:
noteClip = [ int(i) for i in noteClip ]

In [25]:
noteClip.sort()

In [26]:
noteClip

[60]

In [27]:
def diffNote(note_in_song, note_in_src):
    return [x for x in note_in_song if x not in note_in_src]

In [28]:
diff = diffNote(note, noteClip)

In [29]:
diff

[25,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 91,
 92]

In [30]:
def findNearest(array, number):
    return min(array, key=lambda x:abs(x-number))

In [31]:
def synthesis_sound(base, target):
    # config
    tmp_path = './tmp_{}.wav'
    target_path = os.path.join(SRC_PATH, "{}.mp4")
    
    # read video and get audio
    video = VideoFileClip(target_path.format(base))
    audio = video.audio
    
    # write sound tmp
    audio.write_audiofile(tmp_path.format(base))
    
    # read wave file
    y, sr = librosa.load(tmp_path.format(base), sr=16000)
    
    # shift sound
    steps = target - base
    y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=steps)
    
    # write new wave file
    librosa.output.write_wav(tmp_path.format(target), y_shifted, sr)
    
    # read audio
    audio = AudioFileClip(tmp_path.format(target))
    video.audio = audio
    
    # write new video
    video.write_videofile(target_path.format(target))
    
    remove_list = [tmp_path.format(base), tmp_path.format(target)]
    for filePath in remove_list:
        if os.path.exists(filePath):
            os.remove(filePath)

In [33]:
def synthesis_not_found_sound(needed, src):
    diff = diffNote(needed, src)
    
    for d in diff:
        nearest_src = findNearest(src, d)
        synthesis_sound(nearest_src, d)

In [34]:
synthesis_not_found_sound(note, noteClip)

MoviePy - Writing audio in ./tmp_60.wav
MoviePy - Done.


Moviepy - Building video ../new_src\25.mp4.
MoviePy - Writing audio in 25TEMP_MPY_wvf_snd.mp3
MoviePy - Done.

t:   3%|██                                                                   | 6/198 [00:00<00:03, 55.06it/s, now=None]


Moviepy - Writing video ../new_src\25.mp4



Moviepy - Done !
Moviepy - video ready ../new_src\25.mp4


...

Moviepy - Done !
Moviepy - video ready ../new_src\91.mp4


MoviePy - Writing audio in ./tmp_60.wav
MoviePy - Done.


t:   0%|                                                                             | 0/198 [00:00<?, ?it/s, now=None]

Moviepy - Building video ../new_src\92.mp4.
MoviePy - Writing audio in 92TEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video ../new_src\92.mp4



Moviepy - Done !
Moviepy - video ready ../new_src\92.mp4
